In [1]:
import sys
# !{sys.executable} -m pip install seaborn
import itertools
import numpy as np
# import spotipy
import os
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
import spotipy
import spotipy.util
from collections import defaultdict
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import seaborn as sns
%matplotlib inline

# Create your own Spotify app to get the ID and secret.
# https://beta.developer.spotify.com/dashboard/applications
CLIENT_ID = '06fdc33f688440e6bff40f6eb930f21c'
CLIENT_SECRET = 'f83c328423054a73a3daa9ae9045e538'

# Put your Spotify username here.
USERNAME = 'james.hoke.peacock'

REDIRECT_URI = 'http://localhost/'
SCOPE = 'user-library-read playlist-modify-public playlist-modify-private'

# Create a Spotify client that can access my saved song information.
token = spotipy.util.prompt_for_user_token(USERNAME,
                                           SCOPE,
                                           client_id=CLIENT_ID,
                                           client_secret=CLIENT_SECRET,
                                           redirect_uri=REDIRECT_URI)


# token_info = ouath2.

sp = spotipy.Spotify(auth=token)
# Function that returns the next n elements from the iterator. Used because
# Spotify limits how many items you can group into each of its API calls.
def grouper(n, iterable):
    it = iter(iterable)
    while True:
        chunk = tuple(itertools.islice(it, n))
        if not chunk:
            return
        yield chunk
        
# TURN THIS INTO A kmeans training program/evaluation 
# for n playlist uris. It self evaluates how well it can categorize songs 
# into their correct playlist. And it visualizes its learning too
COLORS = ['skyblue', 'orange', 'red']
def build_dataset(playlist_uris):
    uris = set([])
    expected = {} #d efaultdict(list) # if songs are on multiple plists
    color = {}
    clr = 0
    cs = []
    for p in playlist_uris:
        plid = p.split(":")[-1]
        plist =  sp._get("playlists/%s" % (plid))
        track_results = sp._get("playlists/%s/tracks" % (plid), limit=100, offset=0,market='US')
        tracks_on_plist = [track[u'track'][u'uri'] for track in track_results['items']]
        for t in tracks_on_plist:
            expected[t] = plist['name']
            color[t] = COLORS[clr]
        uris = uris.union(set(tracks_on_plist))
        clr += 1
        cs.append(plist['name'])
    # Get the audio features of each of the URIs fetched above.
    uris_to_features = {}
    for group in grouper(50, uris):
        res = sp.audio_features(tracks=group)
        for item in res:
            uris_to_features[item['uri']] = item
        
    return uris_to_features, expected, color, cs


def search_my_playlists(name):
    plists = sp.current_user_playlists()['items']
    for plist in plists:
        if plist['name'] == name:
            print(plist)
            playlist_id = plist['id']
            break

    results = sp.user_playlist(username, playlist_id)['tracks']
    add_uris(results)
    while ('next' in results.keys() and results['next']):
        results = sp.next(results)
        add_uris(results)


FEATURE_VECTOR = [
#     'acousticness',
    'danceability',
#     'duration_ms',
    'energy',
#     'instrumentalness',
#     'key',
#     'liveness',
#     'loudness',
#     'mode',
#     'speechiness',
    'tempo',
#     'time_signature',
#     'valence'
]

def features_to_vector(item):
    return np.array([item[key] for key in FEATURE_VECTOR])

house_home_uri = 'spotify:user:heathrfeathr:playlist:3xwAMNQ6hHChEkG8469Iy1' # deep techno
this_is_tim_uri = 'spotify:user:spotify:playlist:37i9dQZF1DX1PQl5Hsjdb8' # country
this_is_metallica_uri = 'spotify:user:spotify:playlist:37i9dQZF1DXbj9Ksq4BAdj' # rock
african_heat_uri = 'spotify:user:spotify:playlist:37i9dQZF1DWYkaDif7Ztbp' # afro

In [2]:
audio_segments = sp.audio_analysis('spotify:track:0OpcI3rARLsNWgVbPdwHD9')['segments']

In [ ]:
import sklearn.preprocessing
import sklearn.cluster
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

# Given an object with a .transform(), apply it to the data vectors.
def apply_transform(transformer, data):
    return [(x[0], transformer.transform(x[1].reshape(1, -1))) for x in data]

def train_and_apply(transformer, data):
    transformer.fit(data)
    return apply_transform(transformer, data)



# threshold = 0.05 # for binning together. Trying KMeans first
pitch_vecs = [seg['pitches'] for seg in audio_segments]
timbre_vecs = [seg['timbre'] for seg in audio_segments]

def group_vecs(vecs):
    distortions = []
    print(len(vecs), len(vecs[0]))
    K = range(2,12,2)
    X = np.array(vecs)
    for k in K:
        kmeanModel = sklearn.cluster.KMeans(n_clusters=k, n_jobs=-1).fit(X)
        kmeanModel.fit(X)
        distortions.append(sum(np.min(cdist(X, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / X.shape[0])

    #Plot the elbow
    plt.plot(K, distortions, 'bx-')
    plt.xlabel('k')
    plt.ylabel('Distortion')
    plt.title('The Elbow Method showing the optimal k')
    plt.show()

# timbre_scaled = train_and_apply(sklearn.preprocessing.StandardScaler(), np.array(timbre_vecs))
group_vecs(np.array(timbre_vecs))

(1421, 12)
